## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,40.42,88,6,2.33,clear sky
1,1,Butaritari,KI,3.0707,172.7902,81.18,79,32,17.60,scattered clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,44.26,76,40,8.05,scattered clouds
3,3,Mungaa,TZ,-4.9667,34.8833,62.87,60,84,9.31,broken clouds
4,4,Khatanga,RU,71.9667,102.5000,32.47,83,68,6.93,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10) 

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,40.42,88,6,2.33,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,44.26,76,40,8.05,scattered clouds
4,4,Khatanga,RU,71.9667,102.5000,32.47,83,68,6.93,broken clouds
6,6,Severo-Kurilsk,RU,50.6789,156.1250,59.50,95,93,17.25,overcast clouds
8,8,Koslan,RU,63.4564,48.8989,57.74,67,100,4.12,overcast clouds
12,12,Tasiilaq,GL,65.6145,-37.6368,44.76,96,100,20.74,moderate rain
14,14,New Norfolk,AU,-42.7826,147.0587,52.92,71,100,2.24,overcast clouds
17,17,Port Hedland,AU,-20.3167,118.5667,59.05,58,0,6.91,clear sky
20,20,Hokitika,NZ,-42.7167,170.9667,49.10,61,6,4.72,clear sky
22,22,Yulara,AU,-25.2406,130.9889,50.70,52,0,10.04,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

Null values present in the Preferred cities list


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df

183


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,40.42,clear sky,-46.1927,168.8643,
2,Ushuaia,AR,44.26,scattered clouds,-54.8000,-68.3000,
4,Khatanga,RU,32.47,broken clouds,71.9667,102.5000,
6,Severo-Kurilsk,RU,59.50,overcast clouds,50.6789,156.1250,
8,Koslan,RU,57.74,overcast clouds,63.4564,48.8989,
12,Tasiilaq,GL,44.76,moderate rain,65.6145,-37.6368,
14,New Norfolk,AU,52.92,overcast clouds,-42.7826,147.0587,
17,Port Hedland,AU,59.05,clear sky,-20.3167,118.5667,
20,Hokitika,NZ,49.10,clear sky,-42.7167,170.9667,
22,Yulara,AU,50.70,clear sky,-25.2406,130.9889,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")

        

Hotel not found... skipping.8
Hotel not found... skipping.32
Hotel not found... skipping.46
Hotel not found... skipping.49
Hotel not found... skipping.80
Hotel not found... skipping.119
Hotel not found... skipping.132
Hotel not found... skipping.172
Hotel not found... skipping.179
Hotel not found... skipping.206
Hotel not found... skipping.214
Hotel not found... skipping.241
Hotel not found... skipping.251
Hotel not found... skipping.285
Hotel not found... skipping.298
Hotel not found... skipping.309
Hotel not found... skipping.324
Hotel not found... skipping.335
Hotel not found... skipping.339
Hotel not found... skipping.362
Hotel not found... skipping.395
Hotel not found... skipping.435
Hotel not found... skipping.472
Hotel not found... skipping.478
Hotel not found... skipping.490
Hotel not found... skipping.497
Hotel not found... skipping.508
Hotel not found... skipping.524
Hotel not found... skipping.559
Hotel not found... skipping.607
Hotel not found... skipping.652
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,40.42,clear sky,-46.1927,168.8643,Ellie's Villa
2,Ushuaia,AR,44.26,scattered clouds,-54.8000,-68.3000,Albatross Hotel
4,Khatanga,RU,32.47,broken clouds,71.9667,102.5000,"""Khatanga"""
6,Severo-Kurilsk,RU,59.50,overcast clouds,50.6789,156.1250,Kak Doma
8,Koslan,RU,57.74,overcast clouds,63.4564,48.8989,
...,...,...,...,...,...,...,...
638,Ust-Kulom,RU,58.30,clear sky,61.6864,53.6902,Ip Napalkov Vasiliy Dmitriyevich
640,Mnogovershinnyy,RU,47.79,overcast clouds,53.9353,139.9242,Obshchezhitiye 14
652,Sangar,RU,50.56,broken clouds,63.9242,127.4739,
655,Kysyl-Syr,RU,48.06,overcast clouds,63.8986,122.7617,


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))